In [69]:
import glob

import pandas as pd

In [70]:
data_files = glob.glob("*/results/*.csv")

In [71]:
df_github_names_long = pd.concat(
    [ pd.read_csv(fp) for fp in data_files], 
    axis=0, 
    keys=data_files, 
    names=["source", "row"]
).reset_index("source").reset_index(drop=True)
df_github_names_long

,source,ids,uu_user_id,github_user_url
0,github_search/results/ids_search_repos.csv,jonasmoons,NaN,NaN
1,github_search/results/ids_search_repos.csv,gglucass,NaN,NaN
2,github_search/results/ids_search_repos.csv,MarcelRobeer,NaN,NaN
3,github_search/results/ids_search_repos.csv,ekatrukha,NaN,NaN
4,github_search/results/ids_search_repos.csv,UtrechtUniversity,NaN,NaN
...,...,...,...,...
518,pure/results/ids_pure_users.csv,keesmulder,NaN,NaN
519,pure/results/ids_pure_users.csv,RianneSchouten,NaN,NaN
520,pure/results/ids_pure_users.csv,ydluo,NaN,NaN
521,pure/results/ids_pure_users.csv,marnixnaber,NaN,NaN


In [72]:
df_github_names_crosstab = pd.crosstab(df_github_names_long["ids"], df_github_names_long["source"])
df_github_names_crosstab

source,github_search/results/ids_search_repos.csv,github_search/results/ids_search_users.csv,github_search/results/ids_topic_repos.csv,pure/results/ids_pure_users.csv
ids,,,,
19giorgosts,0,1,0,0
4009509,0,1,0,0
AJueling,1,1,0,0
ARYENN,0,1,0,0
AlCorreia,1,0,0,0
...,...,...,...,...
xavierpinho23,0,1,0,0
yaikohi,0,1,0,0
ydluo,0,0,0,1


## Basic analytics

In [73]:
df_github_names_long["github_user_url"].value_counts()

nanoepics                     2
a-quei                        2
velgias                       2
ajinkyakadu125                2
gerkovink                     1
73616e646572                  1
Bovi-analytics                1
chauncyzhu                    1
msdslab                       1
UUDigitalHumanitieslab        1
CLARIAH                       1
kovvalsky                     1
marcelrobeer                  1
lauralwd                      1
kaisakajala                   1
maxvanhaastrecht              1
thomasvdzwaard                1
digitalorientalist            1
UU-Hydro                      1
stefanoschmidt1995            1
uudigitalhumanitieslab        1
kraaijenbrink                 1
RELabUU                       1
hanneoberman                  1
UtrechtRiversEstuaries        1
pimlai                        1
nlpsoc                        1
konstantinoskokos             1
stan-dev                      1
ruudvdent                     1
iv4xr-project>project's       1
jhnienhu

In [74]:
df_github_names_long["uu_user_id"].value_counts()

IVelegrakis         2
GChen               2
SFaez               2
AAKadu              2
SSchmidt            1
LAbzianidze         1
AGStergiou          1
IVKryven            1
KKajala             1
FDalpiaz            1
MMoatsos            1
KKogkalidis         1
SPrins              1
CMPierard           1
GVink               1
JHNienhuis          1
LWCvanLit           1
AGJvandeSchoot      1
DICOCortesOrtuno    1
EHSutanudjaja       1
AMacanovic          1
MILVakar            1
MMHostens           1
LWDijkhuizen        1
MGKleinhans         1
ZWang3              1
JGonggrijp          1
CZhu                1
LMMelgarEstrada     1
TvanderZwaard       1
RJvanderEnt         1
PDAKraaijenbrink    1
SJJSpoel            1
IBoszo              1
AMWegmann           1
YKatrukha           1
CJvanLissa          1
PYolumBirbil        1
MANHaastrecht       1
DPNguyen            1
SWBPrasetya         1
MJRobeer            1
ASobhani            1
HIOberman           1
LDHoitink           1
BARicker  

In [75]:
df_users = df_github_names_long[["ids", "uu_user_id"]].sort_values("uu_user_id").drop_duplicates()

df_users.to_csv("unique_users.csv", index=False)

In [76]:
from ghapi.all import GhApi, pages

api = GhApi()

In [65]:
# df_users["ids"].apply(api.users.get_by_username)

In [68]:
# for x in df_users["ids"].tail(10):
#     api.users.get_by_username(x)

HTTP403ForbiddenError: HTTP Error 403: Forbidden